# 一、使用`__slots__`
正常情况下，当我们定义了一个class实例，可以给它绑定任何属性和方法：

In [1]:
class Student(object):
    pass

In [2]:
s = Student()
s.name = 'Michael'
print(s.name)

Michael


还可以尝试给实例绑定一个方法：

In [3]:
# 定义一个函数作为实例方法
def set_age(self, age):
    self.age = age
    
from types import MethodType
# 给实例绑定一个方法
s.set_age = MethodType(set_age, s)
# 调用实例方法
s.set_age(25)
s.age

25

但是这样个实例绑定的方法对另外一个实例是没有用的。

In [4]:
s2 = Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有的实例都绑定方法，可以给类绑定方法：

In [5]:
def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

In [6]:
s.set_score(100)
s.score

100

In [7]:
s2.set_score(99)
s2.score

99

通常情况下，上面的set_score方法可以直接定义在类中，但是动态绑定允许我们在程序运行过程中给class添加功能。

**使用`__slots__`**<br>
如果我们想要限制实例的属性，比如只允许对`Student`实例添加`name`和`age`属性，可以在定义class的时候，定义一个特殊的`__slots__`变量，来限制能添加的属性：

In [8]:
class Student(object):
    __slots__ = ('name', 'age')

In [9]:
s = Student()
s.name = 'Michael'
s.age = 25

In [10]:
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

由于`score`没有被放到`__slots__`中，所以不能动态绑定。需要注意的是`__slots__`只针对当前类可用，对继承的子类是不起作用的。

# 二、使用`@property`
在绑定属性时，如果我们直接把属性暴露出去，没有办法检查参数，导致属性值被随意修改：

In [12]:
class Student(object):
    pass

def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

s = Student()
s.score = 9999

这样肯定是不行的。为了限制`score`的范围，可以通过一个`set_score()`的方法来设置成绩，并通过`get_score()`来获取成绩，这样就可以进行参数检查：

In [14]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100')
        self._score = value

In [15]:
s = Student()
s.set_score(60)
s.get_score()

60

In [16]:
s.set_score(9999)

ValueError: score must between 0 ~ 100

但是上面的调用略显复杂，没有直接使用属性这么简单。为了简化调用，可以使用内置的`@property`装饰器来把方法变成属性调用：

In [17]:
class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

把一个getter方法变成属性，只需要加上`@property`就可以了，此时`@property`本身又创建了另一个装饰器`@score.setter`，负责把一个setter方法变成属性赋值，于是我们就有了一个可控的属性操作：

In [18]:
s = Student()
s.score = 60
s.score

60

In [19]:
s.score = 9999

ValueError: score must between 0 ~ 100!

还可以定义只读属性，只定义getter方法，不定义setter方法即可：

In [20]:
class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self, value):
        self._birth = value
    
    @property
    def age(self):
        return 2018 - self.birth

上面的`birth`是一个读写属性，但`age`是一个只读属性。

## 练习
请利用`@property`给一个`Screen`对象加上`width`和`height`属性，以及一个只读属性`resolution`：

In [23]:
class Screen(object):
    
    @property
    def width(self):
        return self._width
    
    @width.setter
    def width(self, value):
        if not isinstance(value, int):
            raise ValueError("Screen width must be integer!")
        self._width = value
        
    @property
    def height(self):
        return self._height
    
    @height.setter
    def height(self, value):
        if not isinstance(value, int):
            raise ValueError("Screen height must be integer!")
        self._height = value
        
    @property
    def resolution(self):
        return self._width * self._height

# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!


# 三、多重继承
继承是面向对象的一个重要方式，通过继承，子类就可以扩展父类的功能。

In [24]:
class Animal(object):
    pass

# 大类
class Mammal(Animal):
    pass

class Bird(Animal):
    pass

# 各种动物
class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

从上面可以明显看出类之间的继承关系。现在我们要给动物加上`Runnable`和`Flyable`功能，只需要预先定义好两个类：

In [25]:
class Runnable(object):
    def run(self):
        print("Running....")
        
class Flyable(object):
    def fly(self):
        print("Flying....")

对于需要`Runnable`功能的动物，就继承一个`Runnable`，例如`Dog`：

In [26]:
class Dog(Mammal, Runnable):
    pass

In [27]:
class Bat(Mammal, Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的功能。

**MixIn**<br>
在设计类的继承关系时，通常都是单一继承下来的。例如`Ostrich`继承自`Bird`。但是，如果需要混入额外功能，通过多重继承就可以实现，比如让`Ostrich`同时继承`Bird`和`Runnable`。这种设计被称为“MixIn”

In [29]:
class RunnableMixIn(object):
    def run(self):
        print("Running....")
        
class FlyableMixIn(object):
    def fly(self):
        print("Flying....")

class Ostrich(Bird, RunnableMixIn):
    pass

MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系。

Python自带的很多库也使用了MixIn，例如`TCPServer`和`UDPServer`，同时服务多个用户就必须使用多线程或者多进程，它们由`ForkingMixIn`和`ThreadingMixIn`提供.

# 四、定制类